#Aim

Given a large set of sequences or graphs with ordered vertices find small vertex ordered subsequences that are most discriminative for the set.

Steps:
- devise a negative set
- learn a discriminative model
- annotate importance on vertices
- extract max subarrays 
- cluster them 
 - use fast EDeN string kernel 
 - custering algorithm
 
Output: 
1. all sequence motives in each cluster
2. all initial sequences with motif location (begin,end) and cluster id (build regex from all seqs in cluster and run a find iterator)

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
#code for making artificial dataset
import random
def random_string(length,alphabet_list):
    rand_str = ''.join(random.choice(alphabet_list) for i in range(length))
    return rand_str

def perturb(seed,alphabet_list,p=0.5):
    seq=''
    for c in seed:
        if random.random() < p: c = random.choice(alphabet_list)
        seq += c
    return seq

def make_artificial_dataset(alphabet='ACGU', motives=None, motif_length=6, sequence_length=100, n_sequences=1000, n_motives=2, p=0.2):
    alphabet_list=[c for c in alphabet]
    
    if motives is None:
        motives=[]
        for i in range(n_motives):
            motives.append(random_string(motif_length,alphabet_list))
    else:
        motif_length = len(motives[0])
        n_motives = len(motives)
        
    flanking_length = (sequence_length - motif_length ) / 2
    n_seq_per_motif = n_sequences / n_motives

    counter=0
    seqs=[]
    for i in range(n_seq_per_motif):
        for j in range(n_motives):
            left_flanking = random_string(flanking_length,alphabet_list)
            right_flanking = random_string(flanking_length,alphabet_list)
            noisy_motif = perturb(motives[j],alphabet_list,p)
            seq = left_flanking + noisy_motif + right_flanking
            seqs.append(('>ID%d'%counter,seq))
            counter += 1
    return motives, seqs

In [4]:
from eden.motif import SequenceMotif
help(SequenceMotif)

Help on class SequenceMotif in module eden.motif:

class SequenceMotif(__builtin__.object)
 |  Methods defined here:
 |  
 |  __init__(self, min_subarray_size=7, max_subarray_size=10, min_motif_count=1, min_cluster_size=1, training_size=None, negative_ratio=2, shuffle_order=2, n_iter_search=1, complexity=4, nbits=20, clustering_algorithm=None, n_blocks=2, block_size=None, n_jobs=8, random_state=1)
 |  
 |  fit(self, seqs)
 |  
 |  fit_predict(self, seqs, return_list=False)
 |  
 |  fit_transform(self, seqs, return_match=False)
 |  
 |  load(self, obj)
 |  
 |  predict(self, seqs, return_list=False)
 |      returns for each instance a list with the cluster ids that have a hit
 |      if  return_list=False then just return 1 if there is at least one hit from one cluster.
 |  
 |  save(self, model_name)
 |  
 |  transform(self, seqs, return_match=False)
 |      transform = transform an instance to a dense vector with features as cluster ID and entries 0/1 if a motif is found,
 |      if '

#Experimental Setup

In [5]:
#setup parameters
alphabet='ACGU'
motives=['AAAAAAAAAA','CCCCCCCCCC','GGGGGGGGGG','UUUUUUUUUU']
sequence_length=100
n_sequences=100
p=0.3

#make dataset
motives, seqs = make_artificial_dataset(alphabet=alphabet,motives=motives,sequence_length=sequence_length,n_sequences=n_sequences,p=p)

#display
print 'Motives and sample of their perturbed variants:'
alphabet_list=[c for c in alphabet]
for motif in motives: 
    print
    print motif,
    for i in range(9):
        print perturb(motif,alphabet_list,p=p),

Motives and sample of their perturbed variants:

AAAAAAAAAA AAAAAAAUAG AAAAGAUAAA AAAAAAUAGU GCAAAAAAUA AAAAUAAAUG AAUAAAAAAA AACUAAUAAU AAAAAAAAAU AAAAAAAACG
CCCCCCCCCC CGAACCCCCC CCAACCCCCG GACUCUGCCC CGCCCCCCCC CCCCCCUCGG CCCCCCCCCG CCCCCGCCCC UCCCCCACCA ACACCGCUCU
GGGGGGGGGG GGCGUGGGGG GGGUGGGGGG GGAAGGCGGG GGGUUGGAGU GGCCGGGGCG GGGGGGGUGG GGGGCGGUGG GAGGCCGGGG UGUGUGUCGG
UUUUUUUUUU UUGUUUUUCU UUUUUUAUUU GUUUUGUUUU UAUUUAUUUU UUUUAUUGUU UUUUUCUUUG UUUUUUUUCA UCUUUUUCUU UUUUACUUUU


In [6]:
#save to file
fname='artificial_motif_search_dataset.fa'
with open(fname,'w') as f:
    for header,seq in seqs: 
        f.write(header+"\n")
        f.write(seq+"\n")

In [7]:
from eden.util import configure_logging
import logging
configure_logging(logging.getLogger(),verbosity=2)

In [8]:
%%time
from sklearn.cluster import Birch
ca = Birch(threshold=0.1, n_clusters=4, branching_factor=50)

from eden.motif import SequenceMotif
seqmot = SequenceMotif(training_size=100, complexity=2, nbits=14, clustering_algorithm=ca)
seqmot.fit(seqs)
seqmot.save('seqmot')

model induction: 100 positive instances 7 secs
motives extraction: 67 motives 2 secs
motives clustering: 4 clusters 0 secs
after filtering: 59 motives 4 clusters 0 secs
motif model construction: 0 secs


CPU times: user 2.69 s, sys: 679 ms, total: 3.37 s
Wall time: 10.3 s


In [9]:
for cluster_id in seqmot.motives_db:
    print cluster_id
    for count, motif in sorted(seqmot.motives_db[cluster_id], reverse=True):
        print motif, count

0
GGGGGGGGGG 3
UGUGGGAGG 1
UGUCCUGGG 1
UGGGGGUGA 1
UCCCUGACA 1
GGUGGGGGG 1
GGUGACCAGU 1
GGGGUGUGGA 1
GGGGGGGGGA 1
GACAGGUGUA 1
CUGGGGUGUA 1
CUGAGAGAA 1
CUAGGGGGU 1
CAGGUGACAG 1
AGUGGGGGGG 1
AGGUCCUGAC 1
AGGGUGACA 1
AGGGGGUGA 1
AGAUGUGUGC 1
ACAGUGGGG 1
AAGGUCCAUC 1
1
CCCCCCACCC 2
UGACACCCA 1
UCCUCCCAG 1
UCCCAUCCA 1
CCUCCCCCU 1
CCCUCCCCCC 1
CCCGCCCAC 1
CCCCCCCCU 1
CCCCCCCCCU 1
CCCCCCCCCC 1
CCCCCAUCAC 1
CCACCCCCCC 1
CACCCUCCCA 1
CACCCCCCGC 1
ACCCCCUAC 1
ACCCCCCCCA 1
ACCCACCCCA 1
2
UAAAAAAAAA 2
AAAAAAAAAA 2
UAAAAAACAG 1
UAAAAAAAU 1
AGUAAAAAA 1
ACAUAAAAAU 1
ACAAAAAAAU 1
AACAAUGGGA 1
AAAAAAGGU 1
AAAAAAAAAU 1
AAAAAAAAA 1
3
UUUUUUUUUU 3
UUUUUUUUU 2
UUUUUUUUGU 1
UUUUUUUGU 1
UUUUUUGUA 1
UUUUUGUCC 1
UUUCCUAGG 1
UGUUUUUUUU 1
UGUUUUGGGG 1
GGUAUUUUUU 1


In [10]:
from eden.motif import SequenceMotif
seqmot2 = SequenceMotif()
seqmot2.load('seqmot')

predictions=seqmot2.predict(seqs, return_list=True)
for p in predictions: print p

[2]
[1]
[]
[]
[0]
[1]
[0]
[0]
[2]
[]
[]
[0, 3]
[]
[0, 1]
[]
[3]
[2]
[]
[0]
[3]
[0, 2]
[]
[0]
[3]
[2]
[0, 1]
[]
[3]
[]
[1]
[0]
[0, 3]
[2]
[]
[1]
[]
[0, 2]
[1]
[]
[0, 3]
[]
[1]
[]
[3]
[]
[]
[0]
[0]
[0]
[]
[]
[]
[2]
[1]
[0]
[2, 3]
[]
[1]
[]
[1, 3]
[2]
[]
[]
[]
[2]
[1]
[0]
[]
[]
[1, 3]
[0]
[0]
[2]
[1]
[]
[]
[2]
[1]
[1]
[]
[]
[]
[]
[0]
[]
[0, 1]
[]
[]
[2]
[]
[0]
[]
[]
[]
[]
[]
[2]
[1]
[]
[3]


In [11]:
predictions=seqmot2.predict(seqs, return_list=False)
for p in predictions: print p

1
1
0
0
1
1
1
1
1
0
0
1
0
1
0
1
1
0
1
1
1
0
1
1
1
1
0
1
0
1
1
1
1
0
1
0
1
1
0
1
0
1
0
1
0
0
1
1
1
0
0
0
1
1
1
1
0
1
0
1
1
0
0
0
1
1
1
0
0
1
1
1
1
1
0
0
1
1
1
0
0
0
0
1
0
1
0
0
1
0
1
0
0
0
0
0
1
1
0
1


In [12]:
predictions=seqmot2.transform(seqs, return_match=True)
for p in predictions: print p

[[], [], [(46, 55), (46, 56)], []]
[[], [(44, 54)], [], []]
[[], [], [], []]
[[], [], [], []]
[[(30, 39)], [], [], []]
[[], [(44, 54)], [], []]
[[(45, 55), (46, 56)], [], [], []]
[[(2, 11)], [], [], []]
[[], [], [(46, 56), (47, 56)], []]
[[], [], [], []]
[[], [], [], []]
[[(63, 73)], [], [], [(45, 54), (45, 55), (46, 55)]]
[[], [], [], []]
[[(14, 24)], [(44, 53)], [], []]
[[], [], [], []]
[[], [], [], [(48, 57)]]
[[], [], [(1, 11)], []]
[[], [], [], []]
[[(50, 59)], [], [], []]
[[], [], [], [(45, 54), (45, 55), (46, 55)]]
[[(15, 25)], [], [(47, 56)], []]
[[], [], [], []]
[[(44, 54)], [], [], []]
[[], [], [], [(41, 51), (45, 55), (46, 55), (47, 56)]]
[[], [], [(47, 57), (48, 57)], []]
[[(88, 97)], [(43, 53)], [], []]
[[], [], [], []]
[[], [], [], [(43, 53)]]
[[], [], [], []]
[[], [(45, 55)], [], []]
[[(45, 54)], [], [], []]
[[(86, 96)], [], [], [(45, 54), (45, 55), (46, 55)]]
[[], [], [(44, 53)], []]
[[], [], [], []]
[[], [(24, 33)], [], []]
[[], [], [], []]
[[(87, 97)], [], [(47, 57)],

In [13]:
predictions=seqmot2.transform(seqs, return_match=False)
for p in predictions: print p

[0, 0, 1, 0]
[0, 1, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[1, 0, 0, 0]
[0, 1, 0, 0]
[1, 0, 0, 0]
[1, 0, 0, 0]
[0, 0, 1, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[1, 0, 0, 1]
[0, 0, 0, 0]
[1, 1, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 1]
[0, 0, 1, 0]
[0, 0, 0, 0]
[1, 0, 0, 0]
[0, 0, 0, 1]
[1, 0, 1, 0]
[0, 0, 0, 0]
[1, 0, 0, 0]
[0, 0, 0, 1]
[0, 0, 1, 0]
[1, 1, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 1]
[0, 0, 0, 0]
[0, 1, 0, 0]
[1, 0, 0, 0]
[1, 0, 0, 1]
[0, 0, 1, 0]
[0, 0, 0, 0]
[0, 1, 0, 0]
[0, 0, 0, 0]
[1, 0, 1, 0]
[0, 1, 0, 0]
[0, 0, 0, 0]
[1, 0, 0, 1]
[0, 0, 0, 0]
[0, 1, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 1]
[0, 0, 0, 0]
[0, 0, 0, 0]
[1, 0, 0, 0]
[1, 0, 0, 0]
[1, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 1, 0]
[0, 1, 0, 0]
[1, 0, 0, 0]
[0, 0, 1, 1]
[0, 0, 0, 0]
[0, 1, 0, 0]
[0, 0, 0, 0]
[0, 1, 0, 1]
[0, 0, 1, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 1, 0]
[0, 1, 0, 0]
[1, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 1, 0, 1]
[1, 0, 0, 0]
[1, 0, 0, 0]
[0, 0, 1, 0]
[0, 1, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 1, 0]

In [14]:
%%time
from sklearn.cluster import MiniBatchKMeans
ca = MiniBatchKMeans(n_clusters=4)

from eden.motif import SequenceMotif
seqmot = SequenceMotif(training_size=100, clustering_algorithm=ca)
seqmot.fit(seqs)

for cluster_id in seqmot.motives_db:
    print cluster_id
    for count, motif in sorted(seqmot.motives_db[cluster_id], reverse=True):
        print motif, count

model induction: 100 positive instances 15 secs
motives extraction: 68 motives 5 secs
motives clustering: 4 clusters 0 secs
after filtering: 62 motives 4 clusters 0 secs
motif model construction: 0 secs


0
UUUUUUUUUU 2
UUUUUUUUU 2
UUUUUUUUUG 1
UUUUGUUUUU 1
UUUGUUUGAC 1
GUUUUUUUU 1
GUUUUUUGUA 1
GUAUUUUUUA 1
CUGUUUUUUA 1
AUUUUUUUUG 1
AUUUUUGUC 1
AUAUUUUUG 1
1
CCCCCCACCC 2
UGACACCCAU 1
GUCCCAUAG 1
GCCCGCCCA 1
GCCCCCCCCG 1
GCCCCCCCA 1
CCCUCCCCCC 1
CCCCCCCCCU 1
CCCCCCCCCC 1
ACCCUCCCA 1
ACCCCCCCCU 1
ACCCCCCCCA 1
ACCCACCCCA 1
ACACACUCA 1
2
UAAAAAAAAA 2
AAAAAAAAA 2
UAACAUAAC 1
UAAAUGGAU 1
UAAAGGAAU 1
UAAAAAAAU 1
GUAUAAAAAU 1
GUAAAUAAAA 1
GUAAAAAAU 1
GAAAAAAGGU 1
CAUAAAAAU 1
CAUAAAAAAC 1
CAAAAAAAU 1
AGAAUGAA 1
AAAAAUGAA 1
AAAAAAAAAU 1
AAAAAAAAAG 1
AAAAAAAAAA 1
3
GGGGGGGGGG 2
UGGUGGGGGG 1
UGGGUGGGU 1
UGGGGGUGA 1
UGGGGGGGG 1
UGGAUGGAA 1
GGGGUGUGG 1
GGGGGGGGGA 1
GGGGGGGGG 1
GGAGGGUGAC 1
CUGGGUCCCC 1
CUGAGAGAA 1
CGUGAGAGA 1
CAGUGGGGU 1
AUGUGGGUGC 1
AGUAAGUGGG 1
AGGUAAUUU 1
AGGGGGAGGG 1
CPU times: user 3.52 s, sys: 1.07 s, total: 4.58 s
Wall time: 21.7 s


In [15]:
%%time
from sklearn.cluster import DBSCAN
ca = DBSCAN(eps=0.2, min_samples=3)

from eden.motif import SequenceMotif
seqmot = SequenceMotif(training_size=100, clustering_algorithm=ca)
seqmot.fit(seqs)

for cluster_id in seqmot.motives_db:
    print cluster_id
    for count, motif in sorted(seqmot.motives_db[cluster_id], reverse=True):
        print motif, count

model induction: 100 positive instances 15 secs
motives extraction: 61 motives 5 secs
motives clustering: 5 clusters 0 secs
after filtering: 25 motives 4 clusters 0 secs
motif model construction: 0 secs


0
UAAAAAAAAA 2
AAAAAAAAAA 2
AAAAAAAAA 2
UAAAAAACAG 1
UAAAAAAAU 1
CAAAAAAAU 1
AAAAAAGGU 1
AAAAAAAAAU 1
1
GGGGGGGGGG 2
UGGGGGGGG 1
GGGGGGGGGA 1
GGGGGGGGG 1
2
UUUUUUUUUU 3
UUUUUUUUUG 1
UUUUUUUUU 1
UUUUUUGUUU 1
UUUUGUUUUU 1
UGUUUUUUUU 1
AUUUUUUUUG 1
3
GCCCCCCCCG 1
GCCCCCCCA 1
CCCCCCCCCU 1
CCCCCCCCCC 1
ACCCCCCCCU 1
ACCCCCCCCA 1
CPU times: user 2.89 s, sys: 985 ms, total: 3.88 s
Wall time: 21.3 s
